In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
from get_data import get_data
from api_keys import API_KEY
from data_processing import common_subject, count_subject
from maps import get_map_data, make_geodata, add_map_data

In [ ]:
get_data(116, 117, API_KEY)
bills = pd.read_csv("bills_data.csv")

In [ ]:
bills_per_congress = bills["Congress"].value_counts()
plt.bar(bills_per_congress.index.tolist(), bills_per_congress.tolist())
plt.title("Number of Bills per Congress")
plt.xlabel("Congress")
plt.ylabel("Number of Bills")
plt.locator_params(axis='x', nbins=len(bills_per_congress.index.tolist()))
plt.locator_params(axis='y', nbins=max(bills_per_congress.tolist()))
plt.show()

In [48]:
states_and_policy_areas = bills[["State","Policy Areas"]]
state_policy_dict = {}
for state in states_and_policy_areas["State"].tolist(): 
    state_policy_dict[state] = states_and_policy_areas[states_and_policy_areas["State"]==state]["Policy Areas"].tolist()
state_subject_dict = count_subject(state_policy_dict)
print(state_subject_dict)
most_common_policy_area = common_subject(state_subject_dict)


{'MO': {'Agriculture and Food': 0, 'Animals': 0, 'Armed Forces and National Security': 0, 'Arts, Culture, Religion': 0, 'Civil Rights and Liberties, Minority Issues': 0, 'Commerce': 0, 'Congress': 1, 'Crime and Law Enforcement': 0, 'Economics and Public Finance': 0, 'Education': 0, 'Emergency Management': 0, 'Energy': 0, 'Environmental Protection': 0, 'Families': 0, 'Finance and Financial Sector': 0, 'Foreign Trade and International Finance': 0, 'Government Operations and Politics': 0, 'Health': 0, 'Housing and Community Development': 0, 'Immigration': 0, 'International Affairs': 0, 'Labor and Employment': 0, 'Law': 0, 'Native Americans': 0, 'Public Lands and Natural Resources': 0, 'Science, Technology, Communications': 0, 'Social Sciences and History': 0, 'Social Welfare': 0, 'Sports and Recreation': 0, 'Taxation': 0, 'Transportation and Public Works': 0, 'Water Resources Development': 0}, 'NE': {'Agriculture and Food': 0, 'Animals': 0, 'Armed Forces and National Security': 0, 'Arts, 

{'MO': 'Congress',
 'NE': 'Finance and Financial Sector',
 'MD': 'International Affairs',
 'SC': 'Crime and Law Enforcement',
 'MN': 'Congress',
 'MI': 'Armed Forces and National Security',
 'CA': 'Immigration',
 'FL': 'Emergency Management',
 'MS': 'Public Lands and Natural Resources',
 'PA': 'Government Operations and Politics',
 'TX': 'Crime and Law Enforcement',
 'NC': 'Civil Rights and Liberties, Minority Issues',
 'HI': 'Native Americans',
 'CT': 'Agriculture and Food',
 'NV': 'Agriculture and Food',
 'AK': 'Armed Forces and National Security',
 'WY': 'Health',
 'MT': 'Armed Forces and National Security',
 'IA': 'Crime and Law Enforcement',
 'RI': 'Health',
 'AZ': 'Native Americans',
 'WA': 'Native Americans',
 'AR': 'Public Lands and Natural Resources'}